# Display MLE of Bin Intrinsic Variance

Varying $R_0$ to observe how the MLE of the intrinsic variance in bins changes.  

The plots included in this notebook were generated with the spectroscopic data where using default $R_0$, $z_0$, $U_\odot$, $V_\odot$, $W_\odot$:
   * 0 < r < 15 kpc
   * -1 < z < 1 kpc

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sys

sys.path.append("../gaia_tools/")

import data_analysis
import transformation_constants
import covariance_generation
import data_plot
from import_functions import import_data


In [ ]:
# DATA IMPORT
path = "/hdfs/local/sven/gaia_tools_data/gaia_rv_data_bayes.csv"
data_icrs = import_data(path = path, debug = False)

In [ ]:
# Return BinCollection with MLE estimates

def get_test_coll(r = 8000, num_r_bin = 10, num_z_bin = 4):
    
    theta = (8000, transformation_constants.Z_0, transformation_constants.V_SUN[0][0],transformation_constants.V_SUN[1][0], transformation_constants.V_SUN[2][0])
    v_sun = np.array([[theta[2]], 
                              [theta[3]], 
                              [theta[4]]])
    
    galcen_data = data_analysis.get_transformed_data(data_icrs, 
                                       include_cylindrical = True, 
                                       r_0 = r,
                                       v_sun = v_sun,
                                       debug = False,
                                       is_bayes = True, 
                                       is_source_included = True)

    cov_df = covariance_generation.generate_covmatrices(df = data_icrs, 
                                           df_crt = galcen_data, 
                                           transform_to_galcen = True, 
                                           transform_to_cylindrical = True,
                                           z_0 = theta[1],
                                           r_0 = r,
                                           is_bayes = True,
                                           debug=False)
    
    galcen_data['cov_mat'] = cov_df['cov_mat']


    min_val = np.min(galcen_data.r)
    max_val = np.max(galcen_data.r)
    
    min_val_z = np.min(galcen_data.z)
    max_val_z = np.max(galcen_data.z)
    
    # 3
    bin_collection = data_analysis.get_collapsed_bins(data = galcen_data, 
                                                                 theta = theta, 
                                                                 BL_r_min = min_val - 1, 
                                                                 BL_r_max = max_val + 1, 
                                                                 BL_z_min = -1200, 
                                                                 BL_z_max = 1200, 
                                                                 N_bins = (num_r_bin, num_z_bin),
                                                                 r_drift = False, 
                                                                 debug = False)


    bin_collection.GetMLEParameters()
    
    return bin_collection

---
## Change in summed MLE $\sigma$

This iterates over all the bins at a given $R_0$ and sums together their intrinsic variance.  
The $\sigma_{MLE}$ is not weighted by the amount of stars in the given bin, that is to say,  
the summation below is done over the number of bins in r - z.

In [ ]:
def print_MLE_sigma_bins(r_min, r_max):

    for r_0 in range(r_min, r_max+1, 2000):
        bin_collection = get_test_coll(r_0)
        err_sum =sum([bin.MLE_sigma for bin in bin_collection.bins if bin.MLE_sigma is not None])
        print(r"$R_0$ = {0} --> {1}".format(r_0, err_sum))


In [ ]:
print_MLE_sigma_bins(6000, 12000)

---
## Display Bins

In [ ]:
# Define BinCollection object
bin_collection = get_test_coll(r = transformation_constants.R_0)

In [ ]:
# Plot the bins
data_plot.plot_collapsed_bins(bin_collection, 'source_id', showBinValues = True, mode = 'count')

In [ ]:
# Plot the bins
data_plot.plot_collapsed_bins(bin_collection,
                            'v_phi',
                            showBinValues = True,
                            mode = 'MLE_std')

---